In [1]:
import pandas as pd

bad_lines = []

def bad_line_handler(line):
    bad_lines.append(line)

df = pd.read_csv(
    "/home/ois.ru/mvprokhorova/Desktop/fz.csv",
    sep=',',
    quotechar='"',
    na_values=['\\N'],
    on_bad_lines=bad_line_handler,
    engine='python',
    header=None
)

pd.set_option('display.max_columns', None)

print("Количество строк с ошибками:", len(bad_lines))

Количество строк с ошибками: 78


In [2]:
new_column_names = [f"Column {i+1}" for i in range(df.shape[1])]

df.columns = new_column_names

In [3]:
complete_records_count = df.dropna().shape[0]

total_records_count = df.shape[0]

completion_rate = (complete_records_count/total_records_count) * 100


In [4]:
# Оставляем только нужные колонки в датасете
selected_columns = [
    'Column 3',  # Предполагаем, что это колонка для идентификации
    'Column 12',           # Текстовое описание закупки
    'Column 26',  # Коды ОКПД-2 в текстовом формате
    'Column 27',              # Числовые коды ОКПД-2
    'Column 22',           # Цена контракта
    'Column 9'          # Регион
]

# Переименуем колонки для удобства, если они не совпадают
renamed_columns = {
    'Column 3': 'contract_id',
    'Column 12': 'procurement_object',
    'Column 26': 'okpd_text',
    'Column 27': 'okpd_code',
    'Column 22': 'price',
    'Column 9': 'region'
}

# Оставляем и переименовываем колонки
filtered_data = df[selected_columns].rename(columns=renamed_columns)

filtered_data


,contract_id,procurement_object,okpd_text,okpd_code,price,region
0,211010501746701050100100560015320244,<НЕ ОПРЕДЕЛЕНО>,Услуги почтовой связи и услуги курьерские,53.2,39481.08,Республика Адыгея (Адыгея)
1,211010501832601050100101090012229244,Приобретение пеналов (тубусов) для хранения пе...,Изделия резиновые и пластмассовые,22.2,43500.00,Республика Адыгея (Адыгея)
2,211010501832601050100100310013600244,<НЕ ОПРЕДЕЛЕНО>,Вода природная; услуги по очистке воды и водос...,36.0,67300.00,Республика Адыгея (Адыгея)
3,211010501832601050100100310023600244,<НЕ ОПРЕДЕЛЕНО>,Вода природная; услуги по очистке воды и водос...,36.0,11300.00,Республика Адыгея (Адыгея)
4,211010501862201050100100600013811244,<НЕ ОПРЕДЕЛЕНО>,"Услуги по сбору, обработке и удалению отходов;...",38.1,12181.32,Республика Адыгея (Адыгея)
...,...,...,...,...,...,...
2214629,222753609569575360100104260012120323,Поставка лекарственных препаратов (субвенции) ...,21.2,21.2,883220.00,Забайкальский край
2214630,221380827098038080100100020015310244,"Услуги почтовой связи: прием, обработка, перев...",53.1,53.1,687016.00,Иркутская область
2214631,223503408266750340100102360017112244,Оказание услуг по гидрологическому обследовани...,71.1,71.1,238800.00,Московская область
2214632,222780406281078040100100070070000244,Поставка интерактивного оборудования,26.2,26.2,378333.33,Город Санкт-Петербург город федерального значения


In [5]:
"""filtered_data_4 = filtered_data[
    filtered_data['okpd_code'].astype(str).str.match(r'^(41|42|43)\b') | 
    (filtered_data['okpd_code'].astype(str) == '71.1')
]

# Фильтруем строки с текстовыми значениями в столбце 'коды по ОКПД-2 и иногда они словами'
text_fields = filtered_data_4[pd.to_numeric(filtered_data['okpd_text'], errors='coerce').isna()]

# Создаем соответствие уникальных текстовых значений и их кодов
unique_mappings = text_fields[['okpd_text', 'okpd_code']].drop_duplicates()

# Выводим результат
print(unique_mappings)"""

"filtered_data_4 = filtered_data[\n    filtered_data['okpd_code'].astype(str).str.match(r'^(41|42|43)\x08') | \n    (filtered_data['okpd_code'].astype(str) == '71.1')\n]\n\n# Фильтруем строки с текстовыми значениями в столбце 'коды по ОКПД-2 и иногда они словами'\ntext_fields = filtered_data_4[pd.to_numeric(filtered_data['okpd_text'], errors='coerce').isna()]\n\n# Создаем соответствие уникальных текстовых значений и их кодов\nunique_mappings = text_fields[['okpd_text', 'okpd_code']].drop_duplicates()\n\n# Выводим результат\nprint(unique_mappings)"

In [16]:

SUPERVISION_KEYWORDS = [
    'изыскательные работы', 'изыскательских работ', 'изыскательские работы', 
    'инженерно-изыскательные работы', 'изыскания'
]

SUPERVISION_KEYWORDS_2 = [
    'надзор', 'строительный надзор', 'контроль', 
]

# Список кодов для категории "Прочее"
OTHER_CODES = ['53.2', '22.2', '36.0', '38.1']

# Функция классификации
def classify_group(row):
    okpd_code = str(row['okpd_code'])  # Код ОКПД
    contract_text = str(row['procurement_object']).lower()  # Текст контракта в нижнем регистре
    
    # Проверяем наличие ключевых слов для строительного надзора
    if any(keyword in contract_text for keyword in SUPERVISION_KEYWORDS_2):
        return 'Строительный надзор'

    if any(keyword in contract_text for keyword in SUPERVISION_KEYWORDS):
        return 'ПИР'
        
    # Проверяем, относится ли код к категории "Прочее"
    if okpd_code in OTHER_CODES:
        return 'Прочее'
    
    # Классификация по ОКПД
    if okpd_code.startswith('41') or okpd_code.startswith('42') or okpd_code.startswith('43'):
        if okpd_code.startswith('43.2'):  # Подключение коммуникаций
            return 'Подключение коммуникаций'
        else:  # Все остальное в 41, 42, 43
            return 'СМР'
    

# Применение функции с передачей строки DataFrame
filtered_data['group'] = filtered_data.apply(classify_group, axis=1)


In [17]:
print(filtered_data['group'].value_counts())
# Проверяем, какие коды попали в категорию "Прочее"
unclassified_codes = filtered_data[filtered_data['group'] == 'Прочее']['okpd_code'].unique()
print("Коды, отнесённые к категории 'Прочее':", unclassified_codes)


group
СМР                         147744
Прочее                      113302
Подключение коммуникаций     32173
Строительный надзор           9223
ПИР                            258
Name: count, dtype: int64
Коды, отнесённые к категории 'Прочее': ['53.2' '22.2' '36.0' '38.1']


In [8]:
df_info =  pd.DataFrame({
    "DataType": filtered_data.dtypes,
    "MissingValues": filtered_data.isnull().sum(),
    "UniqueValues": filtered_data.nunique()
}).sort_values(by="MissingValues", ascending = False)

df_info['MissingValuesRatio'] = round(df_info['MissingValues']/len(df),2)

filtered_data.describe(include=['O'])

,contract_id,procurement_object,okpd_text,okpd_code,region,group
count,1838838,2213266,2214634,2214634,2214634,294044
unique,1368111,693977,330,270,89,5
top,182771623753277160100100290030000000,<НЕ ОПРЕДЕЛЕНО>,Продукты пищевые,68.1,Город Москва столица Российской Федерации горо...,СМР
freq,23366,418477,211526,105201,179276,147545


In [9]:
# Filtering the dataset and replacing '<НЕ ОПРЕДЕЛЕНО>' with null (NaN)

# Keeping only the specified columns
filtered_data_5 = filtered_data[['procurement_object', 'okpd_code', 'group']]

# Replacing '<НЕ ОПРЕДЕЛЕНО>' with NaN in the 'procurement_object' column
filtered_data_5['procurement_object'] = filtered_data['procurement_object'].replace('<НЕ ОПРЕДЕЛЕНО>', pd.NA)

# Displaying the updated DataFrame information
df_info = pd.DataFrame({
    "DataType": filtered_data.dtypes,
    "MissingValues": filtered_data_5.isnull().sum(),
    "UniqueValues": filtered_data_5.nunique()
}).sort_values(by="MissingValues", ascending=False)

df_info['MissingValuesRatio'] = round(df_info['MissingValues'] / len(filtered_data_5), 2)
df_info

/tmp/ipykernel_118375/243403179.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data_5['procurement_object'] = filtered_data['procurement_object'].replace('<НЕ ОПРЕДЕЛЕНО>', pd.NA)


,DataType,MissingValues,UniqueValues,MissingValuesRatio
group,object,1920590.0,5.0,0.87
procurement_object,object,419845.0,693976.0,0.19
okpd_code,object,0.0,270.0,0.00
contract_id,object,NaN,NaN,NaN
okpd_text,object,NaN,NaN,NaN
price,float64,NaN,NaN,NaN
region,object,NaN,NaN,NaN


In [10]:
filtered_data_5 = filtered_data_5.dropna()

# Если нужно удалить строки только с пустыми значениями в 'procurement_object'
# filtered_data_4 = filtered_data_4.dropna(subset=['procurement_object'])

# Обновляем информацию о датасете
df_info = pd.DataFrame({
    "DataType": filtered_data_5.dtypes,
    "MissingValues": filtered_data_5.isnull().sum(),
    "UniqueValues": filtered_data_5.nunique()
}).sort_values(by="MissingValues", ascending=False)

df_info['MissingValuesRatio'] = round(df_info['MissingValues'] / len(filtered_data_5), 2)
df_info

,DataType,MissingValues,UniqueValues,MissingValuesRatio
procurement_object,object,0,160341,0.0
okpd_code,object,0,22,0.0
group,object,0,5,0.0


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# Подготовка данных
X = filtered_data_5['procurement_object']
y = filtered_data_5['group']

# Разделение на тренировочные и тестовые данные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Преобразование текста в TF-IDF вектора
vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Обучение модели
model = RandomForestClassifier()
model.fit(X_train_tfidf, y_train)

# Оценка модели
y_pred = model.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))

y_pred = model.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))

# Предсказания на тренировочной выборке
y_train_pred = model.predict(X_train_tfidf)

# Вычисление точности для тренировочной выборки
train_accuracy = accuracy_score(y_train, y_train_pred)

# Предсказания на тестовой выборке
y_test_pred = model.predict(X_test_tfidf)

# Вычисление точности для тестовой выборки
test_accuracy = accuracy_score(y_test, y_test_pred)

# Вывод ошибок
print(f"Точность для тренировочной выборки: {train_accuracy:.2f}")
print(f"Точность для тестовой выборки: {test_accuracy:.2f}")

# Если хотите подробный отчет по метрикам на тестовой выборке
print("\nПодробный отчет по метрикам на тестовой выборке:")
print(classification_report(y_test, y_test_pred))

                          precision    recall  f1-score   support

                     ПИР       0.83      0.59      0.69       123
Подключение коммуникаций       0.84      0.74      0.79      9534
                  Прочее       0.98      0.98      0.98     26125
                     СМР       0.94      0.96      0.95     43278
     Строительный надзор       0.98      0.56      0.72        78

                accuracy                           0.94     79138
               macro avg       0.91      0.77      0.82     79138
            weighted avg       0.94      0.94      0.94     79138

                          precision    recall  f1-score   support

                     ПИР       0.83      0.59      0.69       123
Подключение коммуникаций       0.84      0.74      0.79      9534
                  Прочее       0.98      0.98      0.98     26125
                     СМР       0.94      0.96      0.95     43278
     Строительный надзор       0.98      0.56      0.72        78

     

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import accuracy_score

# Подготовка данных
X = filtered_data_5['procurement_object']
y = filtered_data_5['group']

# Разделение на тренировочные и тестовые данные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Преобразование текста в TF-IDF вектора
vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Список моделей
models = {
    "Random Forest": RandomForestClassifier(),
    "Logistic Regression": LogisticRegression(max_iter=1000)
    #"Gradient Boosting": GradientBoostingClassifier(),
    #"AdaBoost": AdaBoostClassifier(),
    #"Extra Trees": ExtraTreesClassifier(),
    #"Naive Bayes": MultinomialNB(),
    #"Support Vector Machine": SVC(kernel='linear', probability=True),
}

# Обучение и оценка моделей
for model_name, model in models.items():
    print(f"\nМодель: {model_name}")
    
    # Обучение
    model.fit(X_train_tfidf, y_train)
    
    # Предсказания на тренировочной выборке
    y_train_pred = model.predict(X_train_tfidf)
    train_accuracy = accuracy_score(y_train, y_train_pred)
    
    # Предсказания на тестовой выборке
    y_test_pred = model.predict(X_test_tfidf)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    # Вывод точности
    print(f"Точность на тренировочной выборке: {train_accuracy:.2f}")
    print(f"Точность на тестовой выборке: {test_accuracy:.2f}")
    
    # Отчет классификации на тестовой выборке
    print("Отчет классификации на тестовой выборке:")
    print(classification_report(y_test, y_test_pred))



Модель: Random Forest
Точность на тренировочной выборке: 0.99
Точность на тестовой выборке: 0.94
Отчет классификации на тестовой выборке:
                          precision    recall  f1-score   support

                     ПИР       0.82      0.55      0.66       123
Подключение коммуникаций       0.84      0.74      0.79      9534
                  Прочее       0.98      0.98      0.98     26125
                     СМР       0.94      0.96      0.95     43278
     Строительный надзор       0.98      0.67      0.79        78

                accuracy                           0.94     79138
               macro avg       0.91      0.78      0.84     79138
            weighted avg       0.94      0.94      0.94     79138


Модель: Logistic Regression
Точность на тренировочной выборке: 0.93
Точность на тестовой выборке: 0.92
Отчет классификации на тестовой выборке:
                          precision    recall  f1-score   support

                     ПИР       0.78      0.50      0

In [22]:
import gradio as gr
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score

def predict(model_name, text_input):
    model = models[model_name]
    text_tfidf = vectorizer.transform([text_input])  # Преобразование текста в TF-IDF
    prediction = model.predict(text_tfidf)[0]
    return f"Класс: {prediction}"

# Интерфейс Gradio
model_names = list(models.keys())
with gr.Blocks() as interface:
    gr.Markdown("# Классификация текста")
    with gr.Row():
        model_dropdown = gr.Dropdown(choices=model_names, label="Выберите модель")
        text_input = gr.Textbox(lines=3, label="Введите текст для классификации")
    with gr.Row():
        predict_button = gr.Button("Сделать предсказание")
    output_text = gr.Textbox(label="Результат")
    
    # Логика кнопки
    predict_button.click(fn=predict, inputs=[model_dropdown, text_input], outputs=output_text)

# Запуск приложения
interface.launch()


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
